In [1]:
import pandas as pd
import numpy as np

### process different phenotype files

In [2]:
polypred_max1 = pd.read_csv("/gpfs/commons/home/tlin/output/bellenguez/bellenguez_all_2/polypred/PLINKupdate_max1.prs",sep='\t')
polypred_max3 = pd.read_csv("/gpfs/commons/home/tlin/output/bellenguez/bellenguez_all_2/polypred/PLINKupdate_max3.prs",sep='\t')
polypred_max5 = pd.read_csv("/gpfs/commons/home/tlin/output/bellenguez/bellenguez_all_2/polypred/PLINKupdate_max5.prs",sep='\t')
polypred_max7 = pd.read_csv("/gpfs/commons/home/tlin/output/bellenguez/bellenguez_all_2/polypred/PLINKupdate_max7.prs",sep='\t')
polypred_max10 = pd.read_csv("/gpfs/commons/home/tlin/output/bellenguez/bellenguez_all_2/polypred/PLINKupdate_max10.prs",sep='\t')
manifest = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data_10_28_2021/SampleManifest_DS_2021.02.19.v2_ALL.txt", sep='\t') ## the one with mapping info
phenotype1 = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data_10_28_2021/ADNIPhenotypes_DS_2021.02.19.v2_ALL.txt", sep='\t')
phenotype2 = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data_10_28_2021/ADSPCaseControlPhenotypes_DS_2021.02.19.v2_ALL.txt", sep='\t')
phenotype3 = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data_10_28_2021/ADSPFamilyBasedPhenotypes_DS_2021.02.19.v2_ALL.txt", sep='\t', encoding='cp1252')
phenotype4 = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data_10_28_2021/ADSP_phenotype_filtered.tsv", sep='\t')

ADSP_duplicate = pd.read_excel("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data_10_28_2021/gcad.r3.wgs.16905.2021.08.24.pairwise_IBD.summary.xlsx", sep= '\t')
ancestry = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/ADSP_ethnicity_prediction.tsv", sep = "\t")
#phenotype = pd.read_csv("/gpfs/commons/home/tlin/data/phenotype_0219.2021.tsv", sep = '\t')


In [6]:
drop = np.array(ADSP_duplicate[ADSP_duplicate.loc[:,"Notes (suggested duplicate to drop for ADSP focused analyses)"] != "Drop both"]["Notes (suggested duplicate to drop for ADSP focused analyses)"])
drop_both = ADSP_duplicate[ADSP_duplicate.loc[:,"Notes (suggested duplicate to drop for ADSP focused analyses)"] == "Drop both"].loc[:,["SampleID1","SampleID2"]]
drop_both = np.append(np.array(drop_both.SampleID1),np.array(drop_both.SampleID2))
drop_sampleID = np.append(drop_both,drop) ## 306: drop both, 287: suggested to drop

In [7]:
print(drop_sampleID.shape)
print(np.unique(drop_sampleID).shape)
drop_sampleID = np.unique(drop_sampleID)  # there are only 362 unique values

(593,)
(362,)


In [9]:
drop_SUBJID = manifest[manifest.SampleID.isin(drop_sampleID)].SUBJID
drop_SAMPID = manifest[manifest.SampleID.isin(drop_sampleID)].SampleID

In [10]:
print(drop_SUBJID.shape)
print(drop_SAMPID.shape)

(361,)
(361,)


## exploring ancestry file.

In [12]:
print(ancestry.shape)
print(ancestry[ancestry.sample_id.isin(manifest.SampleID)].shape)  ## There is one sample id missing out from manifest file. (Can't find its SUBJID: A-ADC-AD005043-BR-NCR-09AD15795)
print(ancestry[~ancestry.sample_id.isin(manifest.SampleID)])
ancestry = ancestry[ancestry.sample_id.isin(manifest.SampleID)]

(16906, 18)
(16905, 18)
                            sample_id population_pred  max_probability  \
1418  A-ADC-AD005043-BR-NCR-09AD15795             AFR         0.988527   

        prob_0    prob_1    prob_2    prob_3    prob_4    distance  \
1418  0.988527  0.005296  0.002237  0.001738  0.002202  141.635678   

        threshold population_code superpopulation_code super_population_name  \
1418  Not Matched             PUR                  AMR     American Ancestry   

     population_name                     name_concat race_description  \
1418    Puerto Rican  American Ancestry_Puerto Rican            White   

     ethnicity_description final_population  
1418          Not Hispanic              AFR  


In [13]:
ancestry = ancestry[~ancestry.sample_id.isin(drop_SAMPID)]

In [14]:
## add SUBJID to ancestry
ancestry = ancestry.merge(manifest, left_on="sample_id", right_on="SampleID")
ancestry.shape  ## (16544, 28)

(16544, 28)

In [15]:
phenotype1 = phenotype1[phenotype1.SUBJID.isin(ancestry.SUBJID)]
phenotype2 = phenotype2[phenotype2.SUBJID.isin(ancestry.SUBJID)]
phenotype3 = phenotype3[phenotype3.SUBJID.isin(ancestry.SUBJID)]

print(phenotype1.shape)
print(phenotype2.shape)
print(phenotype3.shape)

(1497, 30)
(12286, 27)
(917, 26)


In [16]:
phenotype2[phenotype2.SUBJID.isin(phenotype3.SUBJID)].shape  ## there are overlap within family-based and case control based ADSP phenotypic file. 

(31, 27)

In [18]:
## modified Diagnosis and Age column to the format we want.

phenotype1["Diagnosis"] = phenotype1[['PrevAD', 'IncAD']].max(axis=1).astype(int)
phenotype1['Age'] = np.where(phenotype1.Diagnosis == 1,phenotype1.Age_AD_onset, phenotype1.Age_current)

phenotype2[["PrevAD",'IncAD']] = phenotype2[["PrevAD",'IncAD']].fillna(value = -1)
phenotype2["Diagnosis"] = phenotype2[['PrevAD', 'IncAD']].max(axis=1).astype(int)

np.where(phenotype3.AD.isnull())  # there is no NA in phenotype3
phenotype3["Diagnosis"] = np.where((phenotype3.AD<= 3) & (phenotype3.AD > 0), 1,0)


## add source for further identification 
phenotype1["Source"] = "ADNI"
phenotype2["Source"] = "ADSPCaseControl"
phenotype3["Source"] = "ADSPFamilyBased"

In [19]:
## we cant delete those rows with NAs directly because there might be overlap in other phenotypes that have this information. 
wanted_col = ["SUBJID","Sex","Age","APOE","Race","Diagnosis","Source"]

def check_na(df,name):
    print("phenotye: ",name)
    print("data size: ", df.loc[:,wanted_col].shape[0])
    print("NAs in Diagnosis: ", df[df.Diagnosis == -1].shape[0])
    print("NAs in Age ", df.loc[:,wanted_col].shape[0] -df.loc[:,wanted_col].dropna(subset = ["Age"]).shape[0])
    print("NAs in APOE ", df.loc[:,wanted_col].shape[0] -df.loc[:,wanted_col].dropna(subset = ["APOE"]).shape[0])
    print("NAs in Race ", df.loc[:,wanted_col].shape[0] -df.loc[:,wanted_col].dropna(subset = ["Race"]).shape[0],"\n")

    
check_na(phenotype1,"ADNI Phenotypes")
check_na(phenotype2,"ADSP CaseControlPhenotypes")
check_na(phenotype3,"ADSP FamilyBasedPhenotypes")



phenotye:  ADNI Phenotypes
data size:  1497
NAs in Diagnosis:  0
NAs in Age  6
NAs in APOE  0
NAs in Race  1 

phenotye:  ADSP CaseControlPhenotypes
data size:  12286
NAs in Diagnosis:  727
NAs in Age  1251
NAs in APOE  137
NAs in Race  14 

phenotye:  ADSP FamilyBasedPhenotypes
data size:  917
NAs in Diagnosis:  0
NAs in Age  41
NAs in APOE  5
NAs in Race  3 



In [20]:
phenotype_merge = pd.concat([phenotype1.loc[:,wanted_col],phenotype2.loc[:,wanted_col],phenotype3.loc[:,wanted_col]])
print(phenotype_merge.shape[0])  # phenotype file not omitting Diagnosis  = unknown 
print(phenotype_merge[phenotype_merge.Diagnosis == -1].shape[0])  # unknown cases in the phenotype_merge file

phenotype_merge.loc[phenotype_merge["Age"] == "90+","Age"] = 90

14700
727


In [87]:
# phenotype_merge = phenotype_merge[~phenotype_merge.SUBJID.isin(drop_SUBJID)]
# phenotype_merge.shape  ## removes 222 rows

(14626, 7)

In [23]:
duplicated_SUBJID = phenotype_merge[phenotype_merge.SUBJID.duplicated()].sort_values("SUBJID").SUBJID

In [24]:
phenotype_merge[phenotype_merge.SUBJID.isin(duplicated_SUBJID)].sort_values("SUBJID") ## most of them are duplicates, except for "A-MIA-UM000402", "A-MIA-UM000759","A-MIA-UM009911" (take family based).  A-RAS-RA000011 -> take case control

,SUBJID,Sex,Age,APOE,Race,Diagnosis,Source
11122,A-CUHS-CU000169,1,87,33.0,5.0,0,ADSPCaseControl
2683,A-CUHS-CU000169,1,87,33.0,5.0,0,ADSPFamilyBased
11143,A-CUHS-CU003023,0,86,33.0,5.0,0,ADSPCaseControl
4616,A-CUHS-CU003023,0,86,33.0,5.0,0,ADSPFamilyBased
13272,A-LOAD-LD002055,0,80,33.0,5.0,1,ADSPCaseControl
6111,A-LOAD-LD002055,0,80,33.0,5.0,1,ADSPFamilyBased
10276,A-MIA-UM000037,1,74,34.0,5.0,1,ADSPFamilyBased
27542,A-MIA-UM000037,1,74,34.0,5.0,1,ADSPCaseControl
14893,A-MIA-UM000329,0,73,34.0,5.0,1,ADSPCaseControl
10324,A-MIA-UM000329,0,73,34.0,5.0,1,ADSPFamilyBased


In [40]:
remove = phenotype_merge[(phenotype_merge.SUBJID.isin(duplicated_SUBJID)) & (phenotype_merge.Diagnosis == -1)] 
remove = remove.append(phenotype_merge[(phenotype_merge.SUBJID == "A-RAS-RA000011") & (phenotype_merge.Source ==  "ADSPFamilyBased")])  ## age different in two phenotypic file. Took the younger one because this is a case. 
remove = remove.append(phenotype_merge[(phenotype_merge.SUBJID == "A-MIA-UM000474") & (phenotype_merge.Source ==  "ADSPCaseControl")])  ## age? and also missing APOE and Race


In [50]:
remove = remove.append(phenotype_merge[(phenotype_merge.SUBJID.isin(duplicated_SUBJID)) & (~phenotype_merge.SUBJID.isin(remove.SUBJID)) & (phenotype_merge.Source == "ADSPFamilyBased")])   ## keep all the duplicated ones from "ADSPCaseControl"
print(remove)


                SUBJID  Sex   Age  APOE  Race  Diagnosis           Source
14896   A-MIA-UM000402    0   NaN  34.0   5.0         -1  ADSPCaseControl
14907   A-MIA-UM000759    0   NaN  34.0   5.0         -1  ADSPCaseControl
15781   A-MIA-UM009911    0   NaN  34.0   4.0         -1  ADSPCaseControl
11290   A-RAS-RA000011    1    85  34.0   5.0          1  ADSPFamilyBased
27534   A-MIA-UM000474    1  7383   NaN   NaN          1  ADSPCaseControl
1412    A-MIA-UM000463    1    70  34.0   5.0          1  ADSPFamilyBased
1426    A-MIA-UM000477    1    78  23.0   5.0          1  ADSPFamilyBased
1460    A-MIA-UM000784    0    70  33.0   5.0          1  ADSPFamilyBased
1484    A-MIA-UM000866    1    68  44.0   5.0          1  ADSPFamilyBased
1486    A-MIA-UM000880    1    70  34.0   5.0          1  ADSPFamilyBased
1522    A-MIA-UM001698    0    63  34.0   5.0          1  ADSPFamilyBased
1561    A-MIA-UM009271    1    84  34.0   4.0          1  ADSPFamilyBased
2683   A-CUHS-CU000169    1    87  33.

In [52]:
phenotype_merge = pd.concat([phenotype_merge, remove]).drop_duplicates(keep=False)
phenotype_merge.shape   ## 14669,7

In [85]:
14699-1295

13404

In [94]:
phenotype_merge[phenotype_merge.Diagnosis== -1].shape


(724, 7)

In [84]:
phenotype_merge[['Age','Sex']].isna().sum()

Age    1295
Sex       0
dtype: int64

In [98]:
phenotype_subset.columns

Index(['SUBJID', 'Sex', 'Age', 'APOE', 'Race', 'Diagnosis', 'Source'], dtype='object')

In [101]:
phenotype_subset = phenotype_merge[phenotype_merge.Diagnosis!= -1]
phenotype_subset = phenotype_subset.dropna(how = "any",subset=['Sex', 'Age','APOE'])
phenotype_subset.shape  # 13303, 7

(13303, 7)

In [103]:
print(phenotype_merge.shape)
print(phenotype_subset.shape)

(14669, 7)
(13303, 7)


In [54]:
chirag_all= pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data_10_28_2021/all_phenotypes_unique_ancestry_all_all.tsv",sep = '\t')   ## (14668,32)
chirag_subset = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data_10_28_2021/all_phenotypes_unique_ancestry_subset.tsv", sep = '\t') ## (13265,32)

In [55]:
print(chirag_all.shape)
print(chirag_subset.shape)

(14668, 32)
(13265, 32)


In [105]:
chirag_all.isna().sum()

SUBJID                   0
SampleID                 0
cohort                   0
Sex                      0
APOE                   183
Race                    17
Ethnicity              114
AD_status_final        649
age_covariate         1288
flag_age_covariate       0
Duplicate_SUBJID         0
final_population         0
X1                       0
X2                       0
X3                       0
X4                       0
X5                       0
X6                       0
X7                       0
X8                       0
X9                       0
X10                      0
X11                      0
X12                      0
X13                      0
X14                      0
X15                      0
X16                      0
X17                      0
X18                      0
X19                      0
X20                      0
dtype: int64

In [104]:
chirag_subset.isna().sum()

SUBJID                  0
SampleID                0
cohort                  0
Sex                     0
APOE                    0
Race                   16
Ethnicity             104
AD_status_final         0
age_covariate           0
flag_age_covariate      0
Duplicate_SUBJID        0
final_population        0
X1                      0
X2                      0
X3                      0
X4                      0
X5                      0
X6                      0
X7                      0
X8                      0
X9                      0
X10                     0
X11                     0
X12                     0
X13                     0
X14                     0
X15                     0
X16                     0
X17                     0
X18                     0
X19                     0
X20                     0
dtype: int64

In [72]:
phenotype_merge[~phenotype_merge.SUBJID.isin(chirag_all.SUBJID)]

,SUBJID,Sex,Age,APOE,Race,Diagnosis,Source
27545,A-ADC-AD018987,1,53,33.0,5.0,1,ADSPCaseControl


In [128]:
phenotype2[phenotype2.SUBJID == "A-ADC-AD018987"][['SUBJID', 'Sex', 'PrevAD', 'IncAD', 'Age', 'Age_baseline', 'APOE','Race', 'Ethnicity', 'Selection', 'AD','Comments','Base_AD','Update_ADstatus']]


,SUBJID,Sex,PrevAD,IncAD,Age,Age_baseline,APOE,Race,Ethnicity,Selection,AD,Comments,Base_AD,Update_ADstatus
27545,A-ADC-AD018987,1,1.0,0.0,53,NaN,33.0,5.0,NaN,NaN,1,NaN,1,0


In [107]:
missing.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data_10_28_2021/missing.tsv", sep = "\t", index = False)

In [126]:
missing = phenotype_subset[~phenotype_subset.SUBJID.isin(chirag_subset.SUBJID)]


In [108]:
phenotype_subset[~phenotype_subset.SUBJID.isin(chirag_subset.SUBJID)].shape

(48, 7)

In [135]:
notfound = chirag_subset[~chirag_subset.SUBJID.isin(phenotype_subset.SUBJID)]
notfound

,SUBJID,SampleID,cohort,Sex,APOE,Race,Ethnicity,AD_status_final,age_covariate,flag_age_covariate,...,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20
782,G-ADNI-AN000018,G-ADNI-AN000018-BL-NCR-12AD51740,ADNI,0,34,5.0,0.0,1,68,1,...,2.679468,-0.875629,14.056784,-0.743177,-1.191733,2.524429,2.728702,0.158806,-2.392161,-2.918263
814,G-ADNI-AN000052,G-ADNI-AN000052-BL-NCR-12AD52253,ADNI,1,33,5.0,0.0,1,75,1,...,9.975641,1.208923,15.265370,-0.918264,-0.004410,1.895228,-0.660902,-0.853697,-0.196350,-1.290580
826,G-ADNI-AN000064,G-ADNI-AN000064-BL-NCR-12AD52439,ADNI,0,33,5.0,0.0,1,66,1,...,8.049169,1.793366,13.986545,0.929818,-0.348294,1.866389,1.354152,2.891926,-2.509318,0.584455
923,G-ADNI-AN000170,G-ADNI-AN000170-BL-NCR-12AD57825,ADNI,0,34,5.0,0.0,1,76,1,...,10.763705,1.029194,18.406929,1.405340,-1.069013,0.926140,2.552988,2.148890,-0.260814,0.553339
929,G-ADNI-AN000176,G-ADNI-AN000176-BL-NCR-12AD57974,ADNI,0,34,5.0,0.0,1,83,1,...,8.092003,3.877052,19.638822,-0.605310,-1.305993,2.557488,5.351698,1.495940,-0.497949,4.806965
1211,G-ADNI-AN000478,G-ADNI-AN000478-BL-NCR-08AD6632,ADNI,0,33,5.0,0.0,1,66,1,...,8.692350,2.516670,23.098048,-0.211019,-1.113014,3.189400,2.539009,6.633217,-0.790779,1.951518
1622,A-ACT-AC002160,A-ACT-AC002160-BL-NCR-15AD78687,ADSPCaseControl,0,33,5.0,0.0,0,70,1,...,6.251231,3.805331,13.793354,1.173536,-0.332065,2.096949,1.027683,1.354880,-1.910789,1.507803
7256,A-CUHS-CU010118,A-CUHS-CU010118-BL-COL-45532BL1,ADSPCaseControl,0,34,6.0,1.0,0,74,1,...,-2.781809,1.122462,-6.523222,5.935699,-1.334033,2.951190,5.258292,0.274952,-4.267785,-0.752020
8122,A-LOAD-LD003964,A-LOAD-LD003964-UNK-UNK-UNK0029,ADSPCaseControl,0,33,1.0,0.0,1,90,1,...,2.233884,-0.327136,10.713355,-3.148478,-1.473282,1.318870,1.551802,1.434469,-4.001257,1.572123
11212,A-WCAP-WC000715,A-WCAP-WC000715-BL-COL-36672BL1,ADSPCaseControl,1,33,6.0,1.0,0,77,1,...,-0.936349,-0.684416,0.128599,1.739425,-0.380687,-0.854452,-0.320946,4.035285,-0.161808,1.022249


In [133]:
phenotype1[phenotype1.SUBJID.isin(notfound.SUBJID)]

,SUBJID,Sex,PrevAD,IncAD,Age_current,Age_baseline,Age_MCI_onset,Age_AD_onset,APOE,AUTOPSY,...,Base_AD,Base_MCI,Base_Version,Update_baseline,Update_latest,Update_Diagnosis,Correction,Diagnosis,Age,Source
825,G-ADNI-AN000018,0,0,1,68,60,59,NaN,34,0,...,0,1,ng00067.v5,0,0,0,0,1,NaN,ADNI
859,G-ADNI-AN000052,1,0,1,75,71,70,NaN,33,0,...,0,1,ng00067.v5,0,0,0,0,1,NaN,ADNI
871,G-ADNI-AN000064,0,0,1,66,61,60,NaN,33,0,...,1,0,ng00067.v5,0,0,0,0,1,NaN,ADNI
975,G-ADNI-AN000170,0,0,1,76,73,71,NaN,34,0,...,1,0,ng00067.v5,0,0,0,0,1,NaN,ADNI
981,G-ADNI-AN000176,0,0,1,83,82,80,NaN,34,0,...,0,1,ng00067.v5,0,0,0,0,1,NaN,ADNI
1276,G-ADNI-AN000478,0,0,1,66,62,63,NaN,33,0,...,0,1,ng00067.v5,0,0,0,0,1,NaN,ADNI


In [121]:
phenotype1[phenotype1.SUBJID == 'G-ADNI-AN000052']

,SUBJID,Sex,PrevAD,IncAD,Age_current,Age_baseline,Age_MCI_onset,Age_AD_onset,APOE,AUTOPSY,...,Base_AD,Base_MCI,Base_Version,Update_baseline,Update_latest,Update_Diagnosis,Correction,Diagnosis,Age,Source
859,G-ADNI-AN000052,1,0,1,75,71,70,NaN,33,0,...,0,1,ng00067.v5,0,0,0,0,1,NaN,ADNI


In [146]:
remove

,SUBJID,Sex,Age,APOE,Race,Diagnosis,Source
14896,A-MIA-UM000402,0,NaN,34.0,5.0,-1,ADSPCaseControl
14907,A-MIA-UM000759,0,NaN,34.0,5.0,-1,ADSPCaseControl
15781,A-MIA-UM009911,0,NaN,34.0,4.0,-1,ADSPCaseControl
11290,A-RAS-RA000011,1,85,34.0,5.0,1,ADSPFamilyBased
27534,A-MIA-UM000474,1,7383,NaN,NaN,1,ADSPCaseControl
1412,A-MIA-UM000463,1,70,34.0,5.0,1,ADSPFamilyBased
1426,A-MIA-UM000477,1,78,23.0,5.0,1,ADSPFamilyBased
1460,A-MIA-UM000784,0,70,33.0,5.0,1,ADSPFamilyBased
1484,A-MIA-UM000866,1,68,44.0,5.0,1,ADSPFamilyBased
1486,A-MIA-UM000880,1,70,34.0,5.0,1,ADSPFamilyBased


In [112]:
phenotype2[phenotype2.SUBJID.isin(notfound)].SUBJID

1471      A-ADC-AD000151
1729      A-ADC-AD000492
2004      A-ADC-AD000992
2184      A-ADC-AD001355
2616      A-ADC-AD001990
2774      A-ADC-AD002196
3520      A-ADC-AD003270
3912      A-ADC-AD003885
4845      A-ADC-AD005553
5390      A-ADC-AD006458
5391      A-ADC-AD006460
5529      A-ADC-AD006719
5553      A-ADC-AD006759
8781      A-ADC-AD012708
8792      A-ADC-AD012737
8808      A-ADC-AD012807
8880      A-ADC-AD012990
9321      A-ADC-AD013518
9349      A-ADC-AD013546
9356      A-ADC-AD013554
9359      A-ADC-AD013557
9377      A-ADC-AD013575
9390      A-ADC-AD013589
9391      A-ADC-AD013590
9395      A-ADC-AD013594
9399      A-ADC-AD013598
9435      A-ADC-AD013635
9437      A-ADC-AD013637
9454      A-ADC-AD013654
9530      A-ADC-AD013738
9539      A-ADC-AD013747
9562      A-ADC-AD013770
9580      A-ADC-AD013790
9585      A-ADC-AD013795
9590      A-ADC-AD013801
9598      A-ADC-AD013809
9622      A-ADC-AD013834
9660      A-ADC-AD013873
9802      A-ADC-AD014018
11469    A-CUHS-CU009833


### combine polypred result with phenotypic files

In [40]:
def combine_phenotype_polypred(polypred, phenotype):
    polypred = polypred.merge(phenotype, left_on = "IID", right_on = "SampleID").drop(["IID"],axis=1)
    polypred = polypred.loc[:,["SUBJID","SampleID","FID","PRS","Diagnosis","Sex","Age","APOE","Race"]]
    return(polypred)

In [41]:
polypred_pheno_max1 = combine_phenotype_polypred(polypred_max1, ancestry_merge)
polypred_pheno_max3 = combine_phenotype_polypred(polypred_max3, ancestry_merge)
polypred_pheno_max5 = combine_phenotype_polypred(polypred_max5, ancestry_merge)
polypred_pheno_max7 = combine_phenotype_polypred(polypred_max7, ancestry_merge)
polypred_pheno_max10 = combine_phenotype_polypred(polypred_max10, ancestry_merge)

In [42]:
print("check match/unmatch:")
print("max_SNP = 1, before merging:%s, after merging:%s"%(polypred_max1.shape[0],polypred_pheno_max1.shape[0]) )
print("max_SNP = 3, before merging:%s, after merging:%s"%(polypred_max3.shape[0],polypred_pheno_max3.shape[0]) )
print("max_SNP = 5, before merging:%s, after merging:%s"%(polypred_max5.shape[0],polypred_pheno_max5.shape[0]) )
print("max_SNP = 7, before merging:%s, after merging:%s"%(polypred_max7.shape[0],polypred_pheno_max7.shape[0]) )
print("max_SNP = 10, before merging:%s, after merging:%s"%(polypred_max10.shape[0],polypred_pheno_max10.shape[0]) )


check match/unmatch:
max_SNP = 1, before merging:16906, after merging:14914
max_SNP = 3, before merging:16906, after merging:14914
max_SNP = 5, before merging:16906, after merging:14914
max_SNP = 7, before merging:16906, after merging:14914
max_SNP = 10, before merging:16906, after merging:14914


In [43]:
polypred_pheno_max1.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/diagnosis/prs_diagnosis_0219.2021_max_snp_1.tsv",index=False, sep = '\t')
polypred_pheno_max3.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/diagnosis/prs_diagnosis_0219.2021_max_snp_3.tsv",index=False, sep = '\t')
polypred_pheno_max5.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/diagnosis/prs_diagnosis_0219.2021_max_snp_5.tsv",index=False, sep = '\t')
polypred_pheno_max7.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/diagnosis/prs_diagnosis_0219.2021_max_snp_7.tsv",index=False, sep = '\t')
polypred_pheno_max10.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/diagnosis/prs_diagnosis_0219.2021_max_snp_10.tsv",index=False, sep = '\t') 

In [44]:
polypred_pheno_max1.to_csv("/gpfs/commons/home/tlin/data/prs/bellenguez_all_2/bellenguez_all2_prs_max_snp_1.tsv",index=False, sep = '\t')
polypred_pheno_max3.to_csv("/gpfs/commons/home/tlin/data/prs/bellenguez_all_2/bellenguez_all2_prs_diagnosis_0219.2021_max_snp_3.tsv",index=False, sep = '\t')
polypred_pheno_max5.to_csv("/gpfs/commons/home/tlin/data/prs/bellenguez_all_2/bellenguez_all2_prs_diagnosis_0219.2021_max_snp_5.tsv",index=False, sep = '\t')
polypred_pheno_max7.to_csv("/gpfs/commons/home/tlin/data/prs/bellenguez_all_2/bellenguez_all2_prs_diagnosis_0219.2021_max_snp_7.tsv",index=False, sep = '\t')
polypred_pheno_max10.to_csv("/gpfs/commons/home/tlin/data/prs/bellenguez_all_2/bellenguez_all2_prs_diagnosis_0219.2021_max_snp_10.tsv",index=False, sep = '\t') 

In [72]:
def find_missing(polypred, polypred_pheno):
    missing = polypred[~polypred.IID.isin(polypred_pheno.SampleID)]
    return(missing)

In [44]:
find_missing(polypred_max1, polypred_pheno_max1).to_csv("/gpfs/commons/home/tlin/data/missingID_max1.tsv", sep = '\t')
find_missing(polypred_max3, polypred_pheno_max3).to_csv("/gpfs/commons/home/tlin/data/missingID_max3.tsv", sep = '\t')
find_missing(polypred_max5, polypred_pheno_max5).to_csv("/gpfs/commons/home/tlin/data/missingID_max5.tsv", sep = '\t')
find_missing(polypred_max7, polypred_pheno_max7).to_csv("/gpfs/commons/home/tlin/data/missingID_max7.tsv", sep = '\t')
find_missing(polypred_max10, polypred_pheno_max10).to_csv("/gpfs/commons/home/tlin/data/missingID_max10.tsv", sep = '\t')

In [74]:
find_missing(polypred_max1, polypred_pheno_max1).shape

(1995, 3)